In [ ]:
###Python Packages###

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.colors import LogNorm
import math as mth

import timeit
import Double_Null_utils as dnu
from scipy import optimize
import tables
from mpmath import *


In [ ]:
###Initial Values###

M0=1.0
Q=0.95
Lambda=0.0000
scalarfield=False
vscalarfield=False


In [ ]:
Tol=1
N=10
#Nul=100
#Nup=10**(40)
scal=1
umax=13
vmax=200
datatype=np.float64

rlim=.2/N
siglim=10/N
philim=5/N


#ru0=5.0
#dr0v=.4
ru0=5.0
#dr0v=0.045
dr0u=-.375
dr0v=-1/(4*dr0u)*(1-2*M0/ru0+Q**2/ru0**2)

#dr0v=0.045

bdytype="stan"
scaltostan=False
solveaffine=False

Elist=[1]
#Elist=[1,2,4,8,16]

#uloc=[1/8,1/4,3/8,1/2,5/8]
#uloc=[1/5,2/5,3/5,4/5,9/10]
uloc=[8.2/10]

In [ ]:
###Defining Needed Constants###

scalf=float(scal)
u0=0.0
v0=0.0
rv0=ru0
phiu0=0.0
phiv0=0.0

if M0==0.0:
    du0=1/N
    dv0=1/N    
else:
    du0=M0/N
    dv0=M0/N
    


def fr(r,M0,Q,Lambda):
    return 1-2*M0/r+Q**2.0/r**2.0-Lambda*r**2.0/3.0

if Lambda>0 or Lambda<0:
    sol = optimize.root(fr,[0.687,1.3,50.0],args=(M0,Q,Lambda), method='hybr')
    rminus=sol.x[0]
    rplus=sol.x[1]
    rcosm=sol.x[2]
else:
    rplus=M0+(M0**2-Q**2)**(.5)
    rminus=M0-(M0**2-Q**2)**(.5)
    rcosm=0.0
    

wHD=True 

print("r+: "+str(rplus))
print("r-: "+str(rminus))
print("rc: "+str(rcosm))

###Predicting Run time###

###timer
start = timeit.default_timer()

Nu=N*umax#Nul*umax*100#int(umax/du0)
Nv=int(vmax/dv0)
#print("Number of points for lowest iteration is "+str(Nu*Nv)+","+str(Nu)+"X"+str(Nv))
print("Number of points for highest iteration is "+str(Nu*Nv*max(Elist)**2)+","+str(Nu*max(Elist))+"X"+str(Nv*max(Elist)))

numpoints=0.0
for i in range(0,len(Elist)):
    numpoints=Elist[i]**2.0*Nu*Nv+numpoints
print("Total number of points is "+str(int(numpoints)))

predtime=numpoints/(78000)

print("Runtime: about "+str(format(predtime/60,'.2f'))+" minutes")


In [ ]:
##################################
###Applying Boundary Conditions###

Emax=max([Elist])


rnpf=np.zeros((2,Nu),dtype=datatype)
signpf=np.zeros((2,Nu),dtype=datatype)
phinpf=np.zeros((2,Nu),dtype=datatype)
massnpf=np.full((2,Nu),M0)#np.zeros((2,Nv))
drunpv=np.full((1,Nu-1),-1/(4*dr0v)*(1-2*M0/ru0+Q**2/ru0**2))
 

#write version of boundary for u-boundary
rnpf[0], signpf[0], phinpf[0] = dnu.boundaryu(scal,bdytype,Nu,ru0,dr0u,dv0,umax,M0,Q,Lambda,scalarfield,datatype)


In [ ]:
drunptemp=np.empty((Nv))*np.nan



A=.115
u1=9.5
u2=10

###Applying Propagation Algorithm###

urange=np.arange(0,umax,du0)
dulist=np.full(Nu-1,0)
vrange=np.array([0.0],dtype=datatype)
drvnp=np.array([0.0],dtype=datatype)
dsigvnp=np.array([0.0],dtype=datatype)
dphinpvf=np.array([0.0],dtype=datatype)
massnpf[0][0]=1.0

drvnp[0]=dr0v#-mth.exp(signpf[0][0])/(4.0*dr0v)*(1-2*M0/ru0+Q**2.0/ru0**2.0-Lambda*ru0**2/3)
dsigvnpvalue=100*np.exp(np.nanmax(signpf))
drvnptemp=drvnp[0]
    

i=0
dv=dv0
if M0>0:
    du0=M0/N
elif M0==0:
    du0=1/N

du=du0

if bdytype=="edd" or bdytype=="fulledd":
    bdyvalue=1.0
    dsigvnp[0]=2*(3*Q**2-3*M0*ru0+ru0**4*Lambda)/(ru0*(-3*Q**2+ru0*(6*M0-3*ru0+ru0**3*Lambda)))*drunp[0]
    #dsigunp[0]=2.0*(M0*rnpf[0]-Q**2.0)/(rnpf[0]*(Q**2.0+rnpf[0]*(-2*M0+rnpf[0])))*drunp[0]
else:
    bdyvalue=0.0
    dsigvnp[0]=0.0
 
atom = tables.Float64Atom()

rnpfile = tables.open_file('rnp.h5', mode='w',encoding="utf8") 
rnp = rnpfile.create_earray(rnpfile.root, 'data', atom, (0, Nu))

signpfile = tables.open_file('signp.h5', mode='w',encoding="utf8") 
signp = signpfile.create_earray(signpfile.root, 'data', atom, (0, Nu))
phinpfile = tables.open_file('phinp.h5', mode='w',encoding="utf8") 
phinp = phinpfile.create_earray(phinpfile.root, 'data', atom, (0, Nu))
massnpfile = tables.open_file('massnp.h5', mode='w',encoding="utf8") 
massnp = massnpfile.create_earray(massnpfile.root, 'data', atom, (0, Nu))
    
 
#rtemp=rnpf[0]   
#print(rtemp)

rnp.append(rnpf[0].reshape(1,Nu))
signp.append(signpf[0].reshape(1,Nu))
phinp.append(phinpf[0].reshape(1,Nu))
massnp.append(massnpf[0].reshape(1,Nu))

drvnp=np.append(drvnp,np.nan)
dsigvnp=np.append(dsigvnp,np.nan)
dphinpvf=np.append(dphinpvf,np.nan)

j=0
while vrange[j]<vmax and j<50000:
    i=0
    
    
    #du=du0*Tol*TempTol/max([abs(dsigunpvalue),abs(drunptemp)])
    
    
    
    
    print(vrange[j])
    #print(dv)
    
    #if urange[i]>u1 and urange[i]<u2 and vscalarfield==True:
        #u=urange[i]+du
        #phinpf[1][0]=A*64*(u-u1)**3.0*(u2-u)**3.0/(u2-u1)**6.0
        #dphinpvf[i]=192*A*(u-u1)**2.0*(u-u2)**2.0*(-2*u+u1+u2)/(u1-u2)**6.0 
    #else:
    phinpf[1][0]=0.0
    dphinpvf[j+1]=0.0
    
    #print(drunp,dsigunp,dphinpuf)
    
    
    rnpf[1][0]=rnpf[0][0]+dv*drvnp[j]
    #print(drvnp[j],dsigvnp[j],rnpf[0][0],dphinpvf[j])
    drvnp[j+1]=drvnp[j]+dv*(drvnp[j]*dsigvnp[j]-rnpf[0][0]*dphinpvf[j]**2.0)
    
    
    
    signpf[1][0]=(signpf[0][0]+dv*dsigvnp[j])*(1.0-bdyvalue)+np.log(abs(1.0-2.0*M0/rnpf[1][0]+Q**2.0/rnpf[1][0]**2.0))*bdyvalue
    #dsigunp[i+1]=2.0*(M0*rnpf[1][0]-Q**2.0)/(rnpf[1][0]*(Q**2.0+rnpf[1][0]*(-2*M0+rnpf[1][0])))*drunp[i+1]*bdyvalue
    dsigvnp[j+1]=2*(3*Q**2-3*M0*rnpf[1][0]+rnpf[1][0]**4*Lambda)/(rnpf[1][0]*(-3*Q**2+rnpf[1][0]*(6*M0-3*rnpf[1][0]+rnpf[1][0]**3*Lambda)))*drvnp[j+1]*bdyvalue

    massnpf[1][0]=M0
    
    #print(rnpf[1][0],drvnp[j+1],signpf[1][0],dsigvnp[j+1])
    

    
    #for j in range(0,Nv-1):
    while urange[i]<umax:
        #du=urange[i+1]-urange[i] #from previous line
        try:
            #u1=float(urange[i])
            #u2=float(urange[i+1])
            #du=u2-u1
            du=np.power(2.0,-dulist[i])*du0
            #print(dulist[i])
        except ZeroDivisionError or OverflowError or RuntimeWarning:
            print("Error: du=0")
        except:
            break
        #print(du)
        answer=dnu.x4giveralt(0,i,du,dv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
        #print(answer)
        rnpf[1][i+1]=answer[0]
        phinpf[1][i+1]=answer[1]
        signpf[1][i+1]=answer[2]
        massnpf[1][i+1]=answer[3]
        #drunptemp[j+1]=answer[4]
        drunptemp=answer[4]
        dsigunpvalue=answer[5]
        exterm1=answer[6]
        exterm2=answer[7]
        exterm3=answer[8]
        
        #print(answer)
        #print(rnpf[0][i],rnpf[0][i+1],rnpf[1][i],rnpf[1][i+1])
        
        #print(abs(rnpf[1][i+1]-rnpf[1][i]/rnpf[1][i]),abs(signpf[1][i+1]-signpf[1][i]/signpf[1][i]))
        #if (abs((rnpf[1][i+1]-rnpf[1][i])/rnpf[1][i])>rlim or abs((signpf[1][i+1]-signpf[1][i])/signpf[1][i])>siglim) and i>0:#error too large: #disregrad calculation
        if ((abs(rnpf[1][i+1]-rnpf[1][i]))/abs(rnpf[1][i])>rlim or (abs(signpf[1][i+1])-abs(signpf[1][i]))/abs(signpf[1][i])>siglim) and i>0:  
        #if ((abs(rnpf[1][i+1]-rnpf[1][i]))>rlim or (abs(signpf[1][i+1])-abs(signpf[1][i]))>siglim) and i>0:    
            #print((abs((rnpf[1][i+1]-rnpf[1][i])/rnpf[1][i])>rlim, abs((signpf[1][i+1]-signpf[1][i])/signpf[1][i])>siglim))
            #print(abs((signpf[1][i+1]-signpf[1][i])/signpf[1][i]),siglim)
            #print(signpf[1][i+1],signpf[1][i])
            #keep i same to redo previous calculation
            #insert point in previous function line at i+1 by interpolation
            #rnpf[0]=np.insert(rnpf[0],i+1,dnu.interp(i,rnpf[0],urange))
            #print(dnu.interp(i,rnpf[0],urange))
            
            #print(signpf[0][i],signpf[0][i+1],signpf[0][i+2])
            #print(rnpf[1][i],rnpf[1][i+1],(rnpf[1][i+1]-rnpf[1][i])/rnpf[1][i])
            rnpf=np.insert(rnpf,i+1,dnu.interp(i,rnpf[0],urange,du),axis=1)
            phinpf=np.insert(phinpf,i+1,dnu.interp(i,phinpf[0],urange,du),axis=1)
            signpf=np.insert(signpf,i+1,dnu.interp(i,signpf[0],urange,du),axis=1)
            massnpf=np.insert(massnpf,i+1,dnu.interp(i,massnpf[0],urange,du),axis=1)
            
            #print(signpf[0][i],signpf[0][i+1],signpf[0][i+2])
            
            #print(urange[i-1],urange[i],urange[i+1],urange[i+2])
            
            #rnpf[1]=np.insert(rnpf[1],np.nan,i+1)
            #phinpf[1]=np.insert(phinpf[1],np.nan,i+1)
            #signpf[1]=np.insert(signpf[1],np.nan,i+1)
            #massnpf[1]=np.insert(massnpf[1],np.nan,i+1)
            #temporarily reduce du by a half
            #insert urange[i+1]=urange[i]+du
            #print(rnpf[0][i])
            dulist[i]=dulist[i]+1
            dulist= np.insert(dulist,i,dulist[i])
            urange= np.insert(urange,i+1,urange[i]+du/2)
            #print(urange[i+1])
            #print(du/2)
            #print(rnpf[0][i+1])
            #print(urange[i-1],urange[i],urange[i+1],urange[i+2])
            #record indices of points that will be removed later since they are in between #not needed
            #remlist.append(remlist,i+1)
            #print("Extra point added")
        else:
            #urange[i+1]=urange[i]+du #actually not necessary
            #if saved point is reached, return du to normal and record point # not necessary
            i+=1
        #print(rnpf[0])   
            
            
    
    
    #remove inbetween points when only when recording in order to keep them to copy to f[0,:]
    urange0=np.arange(0,umax,du0)
    indlist=[]
    for i in range(0,len(urange0)):
        indlist.append(urange.tolist().index(urange0[i]))
    
    print(len(indlist),len(rnpf[1]))

    
    rnp.append(np.array([rnpf[1][i] for i in indlist]).reshape(1,Nu))
    signp.append(np.array([signpf[1][i] for i in indlist]).reshape(1,Nu))
    phinp.append(np.array([phinpf[1][i] for i in indlist]).reshape(1,Nu))
    massnp.append(np.array([massnpf[1][i] for i in indlist]).reshape(1,Nu))
    #rnp.append(rnpf[1].reshape(1,Nu))
    #signp.append(signpf[1].reshape(1,Nu))
    #phinp.append(phinpf[1].reshape(1,Nu))
    #massnp.append(massnpf[1].reshape(1,Nu))
        
    drvnp=np.append(drvnp,np.nan)
    dsigvnp=np.append(dsigvnp,np.nan)
    dphinpvf=np.append(dphinpvf,np.nan)
    
    rnpf[0,:]=rnpf[1,:]
    phinpf[0,:]=phinpf[1,:]
    signpf[0,:]=signpf[1,:]
    massnpf[0,:]=massnpf[1,:]
    
    vrange=np.append(vrange,vrange[j]+dv)
 

    j+=1
    
    
    #print(TempTol)
    #print(exterm1)
    #print(exterm2)
    #print(exterm3)
    print('---')
    

    
    
    
    
    if rnpf[1][0]<0.0 or np.isnan(du):
        break
   
    #dumaxlist=[]
    
    
    #if i>1000:
        #break


In [ ]:
print(rnp)

In [ ]:
rnpfile.close()
signpfile.close()
phinpfile.close()
massnpfile.close()

In [ ]:
#Nu=i+2
#print(rnpf[1])

print(Nu)
plt.plot(rnpf[0,:])
plt.plot(rnpf[1,:])
plt.plot([0,len(rnpf[0])],[rplus,rplus],'--')
plt.plot([0,len(rnpf[0])],[rminus,rminus],'--')
#plt.ylim(.8,1.5)
plt.show()

plt.plot(signpf[0,:])
plt.plot(signpf[1,:])
#plt.plot([min(urange),max(urange)],[rplus,rplus])
#plt.plot([min(urange),max(urange)],[rminus,rminus])
plt.show()

plt.plot(urange,rnpf[0,:])
plt.plot(urange,rnpf[1,:])
plt.plot([min(urange),max(urange)],[rplus,rplus],'--')
plt.plot([min(urange),max(urange)],[rminus,rminus],'--')
plt.show()
#rnpf[i+1][j+1]=answer[0]
#phinpf[i+1][j+1]=answer[1]
#signpf[i+1][j+1]=answer[2]
#massnpf[i+1][j+1]=answer[3]
#drunp[i+1][j+1]=answer[4]
#dsigunp[i+1][j+1]=answer[5]

#for i in range(0,len(rnpf[0])):
    #print(rnpf[0][i])

In [ ]:
plt.plot(urange)
plt.show()

In [ ]:
switch=True
np.save('rnputil',np.array([Nu*Nv,Nu,Nv,ru0,dr0v,M0,Q,Lambda,wHD,switch]))
np.save('urange',urange0)
np.save('vrange',vrange)

In [ ]:
stop = timeit.default_timer()

In [ ]:
acttime=stop - start

numpoints=Nu*Nv



print(str(acttime)+' seconds')
print(str((acttime)/60)+' minutes')
print(str((acttime)/3600)+' hours')

print(str(numpoints)+' points')

print(str((numpoints)/(stop - start))+' points per second') 

exttimef=(acttime-predtime)/60

exttime=format(abs((acttime-predtime)/60),'.2f')
if exttimef>0.0:
    print("Took "+str(exttime)+" more minutes")
if exttimef<0.0:
    print("Took "+str(exttime)+" less minutes")
if exttimef==0.0:
    print("Took exactly the right time!!!!")

conv=False
j=0
tempanswer=[]
    while conv==False:
        dv=TempTolv*dv0
        jcount=1
        while j<Nv-1:
            j1=mth.ceil(jcount*TempTolv)
            j2=mth.floor(jcount*TempTolv)
            answer=dnu.x4giveralt(0,j+mth.ceil((jcount-1)*TempTolv),du,dv*TempTolv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
            rnpf[1][j+j1]=answer[0]
            phinpf[1][j+j1]=answer[1]
            signpf[1][j+j1]=answer[2]
            massnpf[1][j+j1]=answer[3]
            #drunptemp[j+1]=answer[4]
            dsigunpv=answer[5]
            jcount=(jcount+1)**(0**j2)
            j=j+j2
        
        if np.abs(tempanswer[0]-answer[0])/answer[0]>0.01 or not tempanswer:
            tempanswer=answer
            TempTol=TempTolv/2
            
        else:
            conv=True
            